In [2]:
using Plots, Random, DataFrames, LinearAlgebra, Printf, JLD, Distributions, Combinatorics, MLJ, Graphs, Gurobi, JuMP, Statistics, Clustering, Distances
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04


Gurobi.Env(Ptr{Nothing} @0x000000015f61bc00, false, 0)

### Generating points

In [3]:
function generate_points(K::Int64, N::Int64, D::Int64 , std::Float64, seed = 1234)
    """
    Generates points for the k-means problem

        input: 
            - I: number of clusters 
            - N: number of points
            - D: dimnesionality of points

        output:
            - points: NxD matrix of standardized points
    """
    Random.seed!(seed);
    X, yy = MLJ.make_blobs(N, D; centers=K, cluster_std=std)
    points = Matrix(DataFrame(X));
    min = minimum(points, dims=1);
    max = maximum(points, dims=1);
    points = (points .- min) ./ (max .- min);
    return points
end

generate_points (generic function with 2 methods)

In [4]:
points = generate_points(2, 20, 2, 0.01);

In [5]:
struct Cluster 
    assignments::Vector{Int64}
    centroid::Vector{Float64}
    cost::Float64
end

In [6]:
function compute_cluster_centroid_cost(assignments::Vector{Int64})
    """
    Computes the centroid and cost of a cluster given the indeces of the points in the cluster

        input: 
            - assignments: Nx1 matrix of cluster assignments

        output:
            - cost of the clustering assignment
            - Vector representing the centroid of the cluster
    """

    cluster_points = points[assignments,:]

    centroid = mean(cluster_points, dims=1)

    return sum( euclidean(cluster_points[i,:],centroid) for i in 1:size(cluster_points,1)), vec(centroid)
end

compute_cluster_centroid_cost (generic function with 1 method)

In [7]:
function compute_reduced_cost(data::Cluster, p::Vector{Float64}, q::Float64)
"""
Computes the reduced cost given a dual solution of the master problem

    input: 
        - Cluster: cluster to compute the reduced cost for
        - p: dual solution of the master problem
        - q: dual solution of the master problem

    output: reduced cost of the cluster
"""

    return data.cost - sum(p[i] for i in data.assignments) - q
end

compute_reduced_cost (generic function with 1 method)

### Subproblem

In [8]:
function new_cluster(data::Matrix{Float64}, p::Vector{Float64}, q::Float64, K::Int64)

    N = size(data,1)
    D = size(data,2)

    #### Modified cost is in the objective 

    ###Subproblem
    sp_model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes( sp_model, "OutputFlag" => 0)

    @variable(sp_model, θ[1:N])
    @variable(sp_model, a[1:N], Bin)
    @variable(sp_model, f[1:N, 1:N])#, Bin)
    @variable(sp_model, b[1:N-K+1], Bin)
    @variable(sp_model, γ[1:N,1:N,1:N-K+1])#, Bin)

    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[i])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[j])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] >= a[i] + a[j] - 1)
    @constraint(sp_model, sum(a[i] for i=1:N) >= 1)

    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= f[i,j])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= b[l])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] >= f[i,j] + b[l] - 1)

    @constraint(sp_model, [i=1:N], [θ[i]; sum( 1/l .* sum( γ[i,j,l] .* (data[i,:] .- data[j,:]) for j=1:N) for l=1:N-K+1)] in SecondOrderCone()) #

    #@constraint(sp_model, [i=1:N],  θ[i] >= sum( sum( 1/l .* sum( γ[i,j,l] .* (data[i,d] .- data[j,d]) for j=1:N) for l=1:N-K+1)^2 for d=1:D) )

    #@constraint(sp_model, [i=1:N, g=1:N], sum(sum( 1/l .* sum( γ[i,j,l] .* (data[i,d] .- data[j,d]) for j=1:N) for l=1:N-K+1) for d=1:D) <= sum(sum( 1/l .* sum( (1-γ[g,j,l]) .* (data[g,d] .- data[j,d]) for j=1:N) for l=1:N-K+1) for d=1:D) )

    @objective(sp_model, Min, sum( θ[i] - a[i]*p[i] for i in 1:N))

    optimize!(sp_model)

    #### Retrieve cluster from solution

    assignments = vec(findall(x->x>0.9, value.(a)))
    @printf("Assignments: %s\n", value.(a))
    #@printf("Assignments: %s\n", value.(θ))
    cost, centroid = compute_cluster_centroid_cost(assignments)
    Clust = Cluster(assignments, centroid, cost)
    reduced_cost = compute_reduced_cost(Clust, p, q)

    return Clust, reduced_cost

end 

new_cluster (generic function with 1 method)

In [55]:
function new_cluster(data::Matrix{Float64}, p::Vector{Float64}, q::Float64, K::Int64, Iter::Int64)

    N = size(data,1)
    D = size(data,2)

    #### Modified cost is in the objective 

    ###Subproblem
    sp_model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes( sp_model, "OutputFlag" => 0)

    @variable(sp_model, θ[1:N])
    @variable(sp_model, a[1:N], Bin)
    @variable(sp_model, f[1:N, 1:N])#, Bin)
    @variable(sp_model, b[1:N-K+1], Bin)
    @variable(sp_model, γ[1:N,1:N,1:N-K+1])#, Bin)

    ####New variable
    @variable(sp_model, z[1:N,1:N-K+1])

    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[i])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] <= a[j])
    @constraint(sp_model, [i=1:N, j=1:N], f[i,j] >= a[i] + a[j] - 1)
    #@constraint(sp_model, sum(a[i] for i=1:N) >= 1)

    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= f[i,j])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] <= b[l])
    @constraint(sp_model, [i=1:N, j=1:N, l=1:N-K+1], γ[i,j,l] >= f[i,j] + b[l] - 1)

    @constraint(sp_model, [i=1:N], [θ[i]; sum( 1/l .* sum( γ[i,j,l] .* (data[i,:] .- data[j,:]) for j=1:N) for l=1:N-K+1)] in SecondOrderCone()) #

    #@constraint(sp_model, [i=1:N],  θ[i] >= sum( sum( 1/l .* sum( γ[i,j,l] .* (data[i,d] .- data[j,d]) for j=1:N) for l=1:N-K+1)^2 for d=1:D) )

    #@constraint(sp_model, [i=1:N, g=1:N], sum(sum( 1/l .* sum( γ[i,j,l] .* (data[i,d] .- data[j,d]) for j=1:N) for l=1:N-K+1) for d=1:D) <= sum(sum( 1/l .* sum( (1-γ[g,j,l]) .* (data[g,d] .- data[j,d]) for j=1:N) for l=1:N-K+1) for d=1:D) )

        #### New contraints for heuristics
    
    @constraint(sp_model, [i=1:N, l=1:N-K+1], z[i,l] >= b[l] + a[i] -1)
    @constraint(sp_model, [i=1:N, l=1:N-K+1], z[i,l] <= b[l])
    @constraint(sp_model, [i=1:N, l=1:N-K+1], z[i,l] <= a[i])

    @constraint(sp_model, [i=1:N], [5/(Iter/10); (data[i,:] .- sum(1/l .* sum( data[j,:] * z[j,l] for j in 1:N)  for l in 1:N-K+1))] in SecondOrderCone())

        #############

    @objective(sp_model, Min, sum( θ[i] - a[i]*p[i] for i in 1:N))

    optimize!(sp_model)


    #### Retrieve cluster from solution

    assignments = vec(findall(x->x>0.9, value.(a)))
    #@printf("Assignments: %s\n", value.(a))
    #@printf("Assignments: %s\n", value.(θ))
    cost, centroid = compute_cluster_centroid_cost(assignments)
    Clust = Cluster(assignments, centroid, cost)
    reduced_cost = compute_reduced_cost(Clust, p, q)

    return Clust, reduced_cost

end 

new_cluster (generic function with 2 methods)

In [56]:
points = generate_points(2, 8, 2, 0.001, 1);
p= [0.000001 , -0.1, 0.000001, -0.000001 , 0.00000001, 0.0000001, 0.00000001, -5.]

new_cluster(points, p , 0.5, 2)

Assignments: [1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0]


(Cluster([1, 3, 5, 6], [0.7498978872817413, 0.25025066085847875], 2.1204416097905083), 1.6204394997905083)

### Master problem

In [57]:
function initial_clusters(data::Matrix{Float64}, K::Int64, n_initial_clusters::Int64)

    clusters = Cluster[]

    for i in 1:n_initial_clusters
        Random.seed!(i)
        km =  kmeans(data', K).assignments
        for k in 1:K
            assinments = findall(x->x==k, km); 
            cost, centroid = compute_cluster_centroid_cost(assinments)
            push!(clusters, Cluster(assinments, centroid, cost))
        end
    end
    return clusters
end

initial_clusters (generic function with 1 method)

In [58]:
function column_generation(data::Matrix{Float64}, K::Int64, n_initial_clusters::Int64, max_iterations::Int = 10000, verbose::Bool = true)

    N = size(data,1)
    
    # Initialization
    CG_solution = nothing
    CG_objective = 0.0
    upper_bounds = []
    lower_bounds = []
    MP_time = []
    SP_time = []
    p_values = []
    q_values = []

    if verbose
        @printf("              | Objective | Lower bound | MP time (s) | SP time (s) | Reduced cost\n")
    end

    # Creates initial clusters
    clusters = initial_clusters(data, K, n_initial_clusters)
    counter = 0

    while true
        counter += 1
        # Make restricted master problem with current clusters 
        MP_start_time = time()
        model = JuMP.Model(() -> Gurobi.Optimizer(GRB_ENV))
        set_optimizer_attributes(
            model,
            "OutputFlag" => 0,
        )

        @variable(model, x[r in 1:length(clusters)] ≥ 0)

        @constraint(model, points_assigned[r=1:N], sum(x[r] for r in 1:length(clusters) if r in clusters[r].assignments) == 1)
        @constraint(model, number_clusters, sum(x[r] for r in 1:length(clusters)) == K)

        @objective(model, Min, sum(x[r] * clusters[r].cost for r in 1:length(clusters)))

        optimize!(model)
        MP_end_time = time()

        ### Get duals
        p = vec(dual.(points_assigned))
        push!(p_values, p)

        q = dual.(number_clusters)
        push!(q_values, q)

        # Update column generation metrics
        push!(upper_bounds, objective_value(model))
        push!(MP_time, MP_end_time - MP_start_time)

        # Compute a new route to add and its corresponding reduced cost
        SP_start_time = time()
        clust, reduced_cost = new_cluster(data, p, q, K, counter)
        SP_end_time = time()

        # Update column generation metrics
        push!(lower_bounds, objective_value(model) + min(N * reduced_cost,0))
        push!(SP_time, SP_end_time - SP_start_time)

        if verbose
            # @printf("              | Objective | MP time (s) | SP time (s) | Reduced cost")
            @printf(
                "Iteration %3d | %9.3f | %11.3f |      %6.3f |      %6.3f | %12.3f\n",
                counter, 
                upper_bounds[end],
                lower_bounds[end],
                MP_time[end],
                SP_time[end],
                reduced_cost,
            )
        end

        # Termination criteria
        if (reduced_cost > -1e-6 || counter > max_iterations)  
            CG_solution = value.(x)
            CG_objective = objective_value(model)
            break
        end
        push!(clusters, clust)

    end

    #### solve MP with integer variables
    ############# TO DO ##################

    return Dict(
        "CG_solution" => CG_solution,
        "CG_objective" => CG_objective,
        "clusters" => clusters,
        "p_values" => p_values,
        "q_values" => q_values,
        "upper_bounds" => upper_bounds,
        "lower_bounds" => lower_bounds,
        "MP_time" => MP_time,
        "SP_time" => SP_time,
        "time_taken" => sum(MP_time) + sum(SP_time),
    )
end

column_generation (generic function with 3 methods)

In [59]:
points = generate_points(2, 20, 2, 0.9);

In [45]:
points = generate_points(2, 20, 2, 0.5);

results = column_generation(points, 2, 4);

              | Objective | Lower bound | MP time (s) | SP time (s) | Reduced cost
Iteration   1 |     1.684 |     -19.340 |       0.005 |       0.406 |       -1.051


Iteration   2 |     1.051 |     -19.972 |       0.001 |       0.342 |       -1.051
Iteration   3 |     1.051 |     -19.972 |       0.001 |       0.377 |       -1.051


Iteration   4 |     1.051 |       1.051 |       0.001 |      10.353 |        3.057
